# Lab 3: Encoders and Decoders

Over the past few weeks, we've been discussing encoders and decoders. This is your chance to computationally implement both an encoder and a decoder. This will be a little bit more computationally challenging, but we know you can do it! We recommend reading up on recursion before doing this lab, as the huffman tree is implemented using recursion. We'll try to make it as easy as possible, while also providing a good learning experience for you.

### Grading Breakdown:
- Problem 1 - 15 Points
- Problem 2 - 15 Points
- Problem 3 - 6 Points
- Problem 4 - 4 Points

Total: 40 Points

### Problem 1: Building Your Encoder

Pedro the racoon is trying to plan Frank the Lizard's birthday party with his friend Gerald the frog. Because of the secret nature of this task, Frank cannot know the contents of the messages. Frank is a highly skilled hacker who unfortunately hasn't taken PHSCS 383, so he doesn't know what an encoding is. You need to build an huffman encoder to assist Pedro in encoding his messages before he sends them to Gerald.

Your task: Build a huffman encoder that encodes based on a seed string. Fill out the following functions:
- __train_encoder() -> HuffmanTree: Trains the huffman encoder, and breaks down the probabilities.
- encode_string(string) -> str: Takes a string and returns the encoded string.


*Important Notes to make your encoder and decoder work with the test cases:*
- Order your nodes first by probability, then by alphabetical order.
- Have the left nodes be associated with 0, and the right nodes be associated with 1.

In [3]:
class TreeNode():
    def __init__(self, identifying_string, probability, left_node, right_node):
        if left_node != None and right_node == None or left_node == None and right_node != None:
            raise("Node should either have two children, or none at all.")

        self.identifier = identifying_string
        self.probability = probability
        self.left_node = left_node
        self.right_node = right_node

    def __str__(self):
        return f"<TreeNode(identifier={self.identifier}, probability={self.probability})>"

    def __repr__(self):
        return f"<TreeNode(identifier={self.identifier}, probability={self.probability})>"


In [4]:
class HuffmanContainer():
    def __init__(self, root_node: TreeNode, encoding_map: dict):
        self.root = root_node
        self.encoding_map = encoding_map

In [5]:
from copy import deepcopy

class Encoder():
    def __init__(self, seed_corpus: str):
        self.seed_corpus = seed_corpus
        self.huffman: HuffmanContainer = self.__train_encoder(seed_corpus)

    def __train_encoder(self, seed_corpus) -> HuffmanContainer:
        symbols = [*seed_corpus]
        unique_symbols = set(symbols)
        frequency_dict = {symbol:0 for symbol in unique_symbols}

        for symbol in symbols:
            frequency_dict[symbol] += 1

        self.probability_dictionary = deepcopy(frequency_dict)
        for symbol in unique_symbols:
            self.probability_dictionary[symbol] = self.probability_dictionary[symbol] / len(symbols)

        root_node: TreeNode = self.__build_tree(self.probability_dictionary)
        self.__symbol_encoding = {key: "" for key in frequency_dict.keys()}
        self.__build_encoding_dict(root_node)

        return HuffmanContainer(root_node, self.__symbol_encoding)

    def __build_tree(self, symbol_probabilities: dict) -> TreeNode:
        tree_nodes = [TreeNode(key, value, None, None) for key, value in symbol_probabilities.items()]
        tree_nodes.sort(key=lambda x: x.probability, reverse=True)

        while True:
            node_one = tree_nodes.pop()
            node_two = tree_nodes.pop()

            combined_node = TreeNode(
                node_one.identifier + node_two.identifier,
                node_one.probability + node_two.probability,
                node_one,
                node_two
            )

            tree_nodes.append(combined_node)

            tree_nodes.sort(key=lambda x: x.probability, reverse=True)

            if len(tree_nodes) == 1:
                break
        root_node = tree_nodes[0]
        return root_node

    def __build_encoding_dict(self, root_node: TreeNode):

        if(root_node.left_node == None and root_node.right_node == None):
            raise Exception("Empty Tree")
        self.__build_encoding_helper("0", root_node.left_node)
        self.__build_encoding_helper("1", root_node.right_node)

    def __build_encoding_helper(self, encoding_key: str, node: TreeNode):
        if(node.left_node != None and node.right_node != None):
            self.__build_encoding_helper(encoding_key + "0", node.left_node)
            self.__build_encoding_helper(encoding_key + "1", node.right_node)
        else:
            self.__symbol_encoding[node.identifier] = encoding_key


    def encode_string(self, string):
        characters_to_encode = [*string]

        encoding_string = []
        for character in characters_to_encode:
            encoding_string.append(self.huffman.encoding_map[character])

        return "".join(encoding_string)

    def get_tree(self):
        return self.huffman


In [ ]:
# Testing Code: Feel free to add as many lines to this code to test your functions above.


In [6]:
# Grading Code: Do not edit this cell.
corpus_string = "abbcaaaaccddabc"
encoder = Encoder(corpus_string)

assert encoder.encode_string("bad") == "1110110"
assert encoder.encode_string("add") == "0110110"
assert encoder.encode_string("cab") == "100111"
print("All Tests Passed!")


All Tests Passed!


Questions to Answer
- What did you learn from building the encoder?
- Did you use recursion? Why or Why not?

*Answer the above question*

### Problem 2: Building Your Decoder
Pedro has successfully encoded his message that he's sending to Gerald. Gerald however has no way to understand what message is being sent. Gerald was sent a packet of decoding information as well as the message. He needs your help to build a decoder to be able to understand the message that Pedro is sending.

Your task: Build a decoder that receives a huffman tree (packet) as an input to it's constructor, and is able to decode the message sent by Gerald.
- decode(encoded_string: str) -> str: Decodes an encoded string.

In [7]:
class Decoder():
    def __init__(self, huffman_tree: HuffmanContainer):
        self.huffman_tree: HuffmanContainer = huffman_tree

    def decode_string(self, encoded_string):
        self.current_index = 0
        self.decoded_string = []
        self.encoded_string = encoded_string

        while True:
            decoded_symbol = self.decode_string_helper(self.huffman_tree.root)
            self.decoded_string.append(decoded_symbol)
            if self.current_index == len(encoded_string):
                break

        return "".join(self.decoded_string)

    def decode_string_helper(self, current_node: TreeNode):
        if current_node.left_node == None and current_node.right_node == None:
            return current_node.identifier

        node_to_traverse = None
        if self.encoded_string[self.current_index] == "0":
            node_to_traverse = current_node.left_node

        if self.encoded_string[self.current_index] == "1":
            node_to_traverse = current_node.right_node

        self.current_index += 1
        return self.decode_string_helper(node_to_traverse)


In [8]:
# Testing Code: Feel free to add as many lines to this code to test your functions above.
decoder = Decoder(encoder.get_tree())

decoder.decode_string("1110110")

'bad'

In [9]:
# Grading Code: Do not edit this cell.
decoder = Decoder(encoder.get_tree())

assert decoder.decode_string("1110110") == "bad"
assert decoder.decode_string("0110110") == "add"
assert decoder.decode_string("100111") == "cab"
print("All Tests Passed!")

All Tests Passed!


### Problem 3: Encoder/Decoder Challenges

This problem will test the robustness of your encoder and decoder. It'll go through a series of tests and see how good your encoder system is at encoding strings.

In [15]:
# Grading Code: Do not edit this cell.
encoder = Encoder("abbaaacdeaacfgashiajkalmmnoppsqrsstttuvwxyz_")

decoder = Decoder(encoder.get_tree())
assert encoder.encode_string("hey_gerald") == "110000110001001111101101110101100011101110100100111101"
assert encoder.encode_string("its_franks_birthday") == "00110101010011011011001111011101110101111000100110110101100011011011110101100001111010100111"
assert encoder.encode_string("lets_give_him_cake") == "00100110001101010011011011101000110111001110001110110110000001100001110110000001111000110001"
assert decoder.decode_string("110000110001001111101101110101100011101110100100111101") == "hey_gerald"
print("All Tests Passed!")

All Tests Passed!


In [23]:
# Grading Code: Do not edit this cell.
encoder = Encoder("aababccdeefffff___")

decoder = Decoder(encoder.get_tree())

assert encoder.encode_string("baaad") == "0100000001100"
assert encoder.encode_string("dab_bad") == "110000010111010001100"
assert encoder.encode_string("dab_fab_ed") == "11000001011110000101110111100"
assert decoder.decode_string("00010111010001100111110000010") == "ab_bad_dab"
print("All Tests Passed!")

All Tests Passed!


In [181]:
# Grading Code: Do not edit this cell.
encoder = Encoder("12435213521235")

decoder = Decoder(encoder.get_tree())

assert encoder.encode_string("111123") == "0000000011101"
assert encoder.encode_string("54321") == "011001011100"
assert encoder.encode_string("1234") == "0011101100"
assert decoder.decode_string("0011101100") == "1234"
print("All Tests Passed!")

All Tests Passed!


Now finally, decode this special message!

In [25]:
encoder = Encoder("abbaaacdeaacfgashiajkalmmnoppsqrsstttuvwxyz_!")
decoder = Decoder(encoder.get_tree())
special_message = "1100111111111110101001100011000001111010111101100111101100011011110110011111110111101010101100011100111101111010000110011111011101010000010011111101010011000110000011110101111011100111111011110100111101000000110010100111011010111010111000001101000110000"

hzwrlywxomlxoxwt_hswcossaiowrlywxhxwawifoatwvlpe


### Problem 4: Write Up

Your task: Answer the following questions
1. What were some things you learned from this lab?
2. What did you like about this lab?
3. What would you improve?

If the student answers all the questions with a good amount of effort, give them full credit!